In [7]:
import pandas as pd
import numpy as np

from scipy.stats import wilcoxon
from statsmodels.sandbox.stats.multicomp import multipletests 

import scipy

In [8]:
aucs = pd.read_csv('AUCs.txt', sep = '\t')
aucs.head()

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


In [25]:
data_list = ['C4.5','C4.5+m','C4.5+cf', 'C4.5+m+cf']
corr_data = []

for i, key1 in enumerate(data_list):
    for j, key2 in enumerate(data_list):
        if i >= j:
            continue
        
        corr, p = stat, p = wilcoxon(aucs[key1], aucs[key2])
        corr_data.append([key1, key2, corr, p])

aucs_correlation = pd.DataFrame.from_records(corr_data)
aucs_correlation.columns = ['aucs_A', 'aucs_B', 'corr', 'p']
aucs_correlation.head()  

,aucs_A,aucs_B,corr,p
0,C4.5,C4.5+m,6.5,0.010757
1,C4.5,C4.5+cf,43.0,0.861262
2,C4.5,C4.5+m+cf,11.0,0.015906
3,C4.5+m,C4.5+cf,17.0,0.046333
4,C4.5+m,C4.5+m+cf,22.0,0.327826


In [26]:
min_p = 1
min_key_1 = ''
min_key_2 = ''
count = 0
for key1, key2, corr, p in corr_data:
    if p < 0.05:
        count+= 1
    if p < min_p:
        min_p = p
        min_key_1 = key1
        min_key_2 = key2
print(min_key_1, min_key_2)    



C4.5 C4.5+m


In [27]:
print(count)  

4


In [28]:
reject, p_corrected, a1, a2 = multipletests(aucs_correlation.p, 
                                            alpha = 0.05, 
                                            method = 'holm') 

In [29]:
aucs_correlation['p_corrected'] = p_corrected
aucs_correlation['reject'] = reject
aucs_correlation.head()

,aucs_A,aucs_B,corr,p,p_corrected,reject
0,C4.5,C4.5+m,6.5,0.010757,0.064543,False
1,C4.5,C4.5+cf,43.0,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,11.0,0.015906,0.079532,False
3,C4.5+m,C4.5+cf,17.0,0.046333,0.138998,False
4,C4.5+m,C4.5+m+cf,22.0,0.327826,0.655651,False


In [30]:
aucs_correlation.reject.value_counts()

False    6
Name: reject, dtype: int64

In [31]:
reject, p_corrected, a1, a2 = multipletests(aucs_correlation.p, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 
aucs_correlation['p_corrected'] = p_corrected
aucs_correlation['reject'] = reject
aucs_correlation.head()


,aucs_A,aucs_B,corr,p,p_corrected,reject
0,C4.5,C4.5+m,6.5,0.010757,0.045818,True
1,C4.5,C4.5+cf,43.0,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,11.0,0.015906,0.045818,True
3,C4.5+m,C4.5+cf,17.0,0.046333,0.069499,False
4,C4.5+m,C4.5+m+cf,22.0,0.327826,0.393391,False


In [32]:
aucs_correlation.reject.value_counts()

True     3
False    3
Name: reject, dtype: int64